In [22]:
import os
import skimage.io
from skimage import transform
from skimage.color import rgb2gray
import numpy as np

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

Load the data

In [3]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".ppm")]
        for f in file_names:
            images.append(skimage.io.imread(f))
            labels.append(int(d))
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [4]:
images, labels = load_data("Training")

Resize the images to 28x28 pixels  
Convert the images to grayscale  
Reshape to 28x28x1 (instead of 28x28)

In [17]:
def transform_images(images):
    images28 = [transform.resize(image, (28,28)) for image in images]
    images28 = np.array(images28)
    images28 = rgb2gray(images28)
    images28 = images28.reshape(-1, 28, 28, 1)
    return images28

In [18]:
images28 = transform_images(images)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Train test split

In [19]:
X_train, X_val, y_train, y_val = train_test_split(images28, labels, test_size=0.20, random_state=42)

One-hot the labels (make into a 2D array of 0's and 1's, rather than a 1D array of categories)

In [20]:
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)

In [21]:
X_train.shape,X_val.shape,y_train_one_hot.shape,y_val_one_hot.shape

((3660, 28, 28, 1), (915, 28, 28, 1), (3660, 62), (915, 62))

The images are of size 28 x 28. You convert the image matrix to an array, rescale it between 0 and 1, reshape it so that it's of size 28 x 28 x 1, and feed this as an input to the network. You'll use three convolutional layers:  
The first layer will have 32-3 x 3 filters,  
The second layer will have 64-3 x 3 filters and  
The third layer will have 128-3 x 3 filters.  
In addition, there are three max-pooling layers each of size 2 x 2.

In [24]:
batch_size = 64
epochs = 20
num_classes = 62